<a href="https://colab.research.google.com/github/Sompote/SPT_Bangkok_SAND/blob/main/SPT_train_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Import libraries
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import pandas as pd
import keras



df = pd.read_excel('SPT.xlsx')
data=np.array(df)
X=data[:,0:2]
y=data[:,2]
y=np.reshape(y,(X.shape[0],1))

# Load and preprocess data
from sklearn.model_selection import KFold
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
sc = MinMaxScaler()
sc_y=MinMaxScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
y_train=sc_y.fit_transform(y_train)
y_test=sc_y.transform(y_test)
def mape(actual, pred):
  return np.mean(np.abs((actual - pred) / actual)) * 100

# Define neural network model
model = Sequential()
model.add(Dense(100, activation='relu', input_dim=X_train.shape[1]))
model.add(Dense(50, activation='relu'))
model.add(Dense(5, activation='relu'))


model.add(Dense(y_train.shape[1]))

# Compile model
model.compile(optimizer='adam', loss='mean_squared_error')

k = 8 # number of folds kf = KFold(n_splits=k, shuffle=True, random_state=42)
kf = KFold(n_splits=k, shuffle=True, random_state=42)
mse_list = []
mape_list = []
for train_index, test_index in kf.split(X_train):
  X_train_fold, X_test_fold = X_train[train_index],X_train[test_index]
  y_train_fold, y_test_fold = y_train[train_index], y_train[test_index]
  model.fit(X_train_fold, y_train_fold, batch_size=5, epochs=500,verbose=0)
  y_pred_fold = model.predict(X_test_fold)
  mse_fold = model.evaluate(X_test_fold, y_test_fold)
  mape_fold = mape(y_test_fold, y_pred_fold)
  mse_list.append(mse_fold)
  mape_list.append(mape_fold)
  print('mean absolute percentage error:', mape_fold)
mse_avg = np.mean(mse_list)
mape_avg = np.mean(mape_list)

1/1 [==============================] - 0s 124ms/step - loss: 0.0055
mean absolute percentage error: 9.249404784399546
1/1 [==============================] - 0s 24ms/step - loss: 0.0078
mean absolute percentage error: inf


<ipython-input-2-0213273b29b0>:30: RuntimeWarning: divide by zero encountered in true_divide
  return np.mean(np.abs((actual - pred) / actual)) * 100


1/1 [==============================] - 0s 23ms/step - loss: 0.0051
mean absolute percentage error: 9.640417978523065
1/1 [==============================] - 0s 22ms/step - loss: 0.0038
mean absolute percentage error: 19.20486832264044
1/1 [==============================] - 0s 23ms/step - loss: 0.0011
mean absolute percentage error: 2.978753160229374
1/1 [==============================] - 0s 23ms/step - loss: 5.5044e-04
mean absolute percentage error: 11.241484053850087
1/1 [==============================] - 0s 23ms/step - loss: 6.3129e-04
mean absolute percentage error: 13.614616582149255
1/1 [==============================] - 0s 39ms/step - loss: 5.6492e-05
mean absolute percentage error: 1.6070000715448316


In [3]:
# plot and save your model
import tensorflow as tf
img_file = 'model.png'
tf.keras.utils.plot_model(model, to_file=img_file, show_shapes=True, show_layer_names=True)
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 100)               300       
                                                                 
 dense_7 (Dense)             (None, 50)                5050      
                                                                 
 dense_8 (Dense)             (None, 5)                 255       
                                                                 
 dense_9 (Dense)             (None, 1)                 6         
                                                                 
Total params: 5,611
Trainable params: 5,611
Non-trainable params: 0
_________________________________________________________________
None


In [4]:
import pickle
model.save('model.h5')
pickle.dump(sc, open('scaler.pkl', 'wb'))
pickle.dump(sc_y, open('scaler_y.pkl', 'wb'))